In [1]:

import tkinter as tk
from tkinter.ttk import *
import cv2
from PIL import Image, ImageTk
from tkinter import filedialog
import pandas as pd
from ultralytics import YOLO
import cvzone
import threading

import face_recognition
import cv2
import tensorflow as tf
import pandas as pd

import cv2
import glob
import numpy as np
import os
import face_recognition
from datetime import datetime
import streamlit as st
from streamlit_webrtc import webrtc_streamer, VideoTransformerBase


# Global variables for OpenCV-related objects and flags
cap = None
is_camera_on = False




path = r"C:\Users\zbook 17 g3\Desktop\CV PROJECTS\New folder (4)"
images = []
classNames = []
myList = os.listdir(path)
print(myList)



for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)



def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList



def add_mark(name) :
      # استيراد مكتبة openpyxl

# اسم ملف الإكسل
    excel_file_path = r"E:\project\sheet 111.xlsx"
# اسم القيمة التي ترغب في البحث عنها
    target_value = name

# قراءة ملف الإكسل إلى DataFrame باستخدام مكتبة openpyxl
    df = pd.read_excel(excel_file_path, engine='openpyxl')

# التحقق من وجود القيمة المستهدفة في العمود المناسب (قم بتغيير اسم العمود حسب ملف الإكسل الخاص بك)
    if target_value in df['data'].values:
    # إذا وجدت القيمة، قم بتعديل العمود المقابل بقيمة 1
        df.loc[df['data'] == target_value, 'value'] = 1

# حفظ DataFrame في ملف الإكسل باستخدام مكتبة openpyxl
    df.to_excel(excel_file_path, index=False, engine='openpyxl')

encodeListKnown = findEncodings(images)
print('Encoding Complete')





# Function to start the webcam feed
def start_webcam():
    global cap, is_camera_on
    if not is_camera_on:
        cap = cv2.VideoCapture(0)  # Use the default webcam (you can change the index if needed)
        is_camera_on = True
        update_canvas()  # Start updating the canvas

# Function to stop the webcam feed
def stop_webcam():
    global cap, is_camera_on
    if cap is not None:
        cap.release()
        is_camera_on = False
        



# Function to update the Canvas with the webcam frame or video frame
def update_canvas():
    global is_camera_on
    if is_camera_on:
        success, img = cap.read()
        if not success:
            print("Failed to read a frame from the camera.")
            return
        imgS = cv2.resize(img, (0, 0), fx=0.25, fy=0.25)
        imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
        facesCurFrame = face_recognition.face_locations(imgS)
        encodesCurFrame = face_recognition.face_encodings(imgS,facesCurFrame)
        for encodeFace,faceLoc in zip(encodesCurFrame,facesCurFrame):
            matches = face_recognition.compare_faces(encodeListKnown,encodeFace)
            print('matches',matches)
            faceDis = face_recognition.face_distance(encodeListKnown,encodeFace)
            print(faceDis)
            matchIndex = np.argmin(faceDis)
            if matches[matchIndex]:
                name = classNames[matchIndex].upper()
                print(name)
                faceLoc = np.array(faceLoc)
                faceLoc = faceLoc / 0.25
                faceLoc=faceLoc.astype(int)
            #y1,x2,y2,x1 = faceLoc # in the other code we risize the face
            #y1, x2, y2, x1 = y1*4,x2*4,y2*4,x1*4
                y1, x2, y2, x1 = faceLoc[0], faceLoc[1], faceLoc[2], faceLoc[3]
         
                cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
                cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
                cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
                
        
                add_mark(name)
 
        frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (520, 380))

        photo = ImageTk.PhotoImage(image=Image.fromarray(img))
        canvas.img = photo
        canvas.create_image(0, 0, anchor=tk.NW, image=photo)

    canvas.after(1, update_canvas)


# Function to quit the application
def quit_app():
    stop_webcam()
    root.quit()
    root.destroy()

# Create the main Tkinter window
root = tk.Tk()
root.title("Attendance system")

# Create a Canvas widget to display the webcam feed or video
canvas = tk.Canvas(root, width=600, height=400)
canvas.pack(fill='both', expand=True)


button_frame = tk.Frame(root)
button_frame.pack(fill='x')

play_button = tk.Button(button_frame, text="Play", command=start_webcam)
play_button.pack(side='left')

stop_button = tk.Button(button_frame, text="Stop", command=stop_webcam)
stop_button.pack(side='left')


quit_button = tk.Button(button_frame, text="Quit", command=quit_app)
quit_button.pack(side='left')

initial_image = Image.open(r"C:\Users\zbook 17 g3\Desktop\HOG\WALLPAPER.jpg")  # Replace 'initial_image.jpg' with your image path
initial_photo = ImageTk.PhotoImage(image=initial_image)
canvas.img = initial_photo
canvas.create_image(0, 0, anchor=tk.NW, image=initial_photo)

root.mainloop()

['Messi.jpg', 'Puten.jpeg', 'tony soprano.jpg']
['Messi', 'Puten', 'tony soprano']
Encoding Complete
